In [48]:
import pandas as pd
import numpy as np


In [49]:
df = pd.read_csv('/content/tmdb_5000_movies.csv')

In [50]:
df.shape

(4803, 20)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [52]:
df= df.drop(['budget','homepage','id','keywords','original_language','original_title','popularity','production_companies','production_countries','release_date','revenue','runtime','spoken_languages','status','tagline','vote_average','vote_count'],axis=1)

In [53]:
df.shape

(4803, 3)

In [54]:
df.head()

,genres,overview,title
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...",Avatar
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",A cryptic message from Bond’s past sends him o...,Spectre
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",Following the death of District Attorney Harve...,The Dark Knight Rises
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","John Carter is a war-weary, former military ca...",John Carter


In [55]:
df['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [56]:
from ast import literal_eval
import json

In [57]:
def json_to_arr(cell, wanted = "name"):
    cell = literal_eval(cell)
    if cell == [] or (isinstance(cell, float) and cell.isna()):
        return np.nan
    result = []
    counter = 0
    for element in cell:
        if counter < 3:
            result.append(element[wanted])
            counter += 1
        else:
            break
    return result[:3]

In [58]:
df[['genres']] = df[['genres']].applymap(json_to_arr)

In [59]:
df['genres']

0               [Action, Adventure, Fantasy]
1               [Adventure, Fantasy, Action]
2                 [Action, Adventure, Crime]
3                     [Action, Crime, Drama]
4       [Action, Adventure, Science Fiction]
                        ...                 
4798               [Action, Crime, Thriller]
4799                       [Comedy, Romance]
4800                [Comedy, Drama, Romance]
4801                                     NaN
4802                           [Documentary]
Name: genres, Length: 4803, dtype: object

In [60]:
df.head()

,genres,overview,title
0,"[Action, Adventure, Fantasy]","In the 22nd century, a paraplegic Marine is di...",Avatar
1,"[Adventure, Fantasy, Action]","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End
2,"[Action, Adventure, Crime]",A cryptic message from Bond’s past sends him o...,Spectre
3,"[Action, Crime, Drama]",Following the death of District Attorney Harve...,The Dark Knight Rises
4,"[Action, Adventure, Science Fiction]","John Carter is a war-weary, former military ca...",John Carter


In [61]:
df['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

Now Text pre-processing

1.Lowercasing

In [62]:
df['overview'].str.lower()

0       in the 22nd century, a paraplegic marine is di...
1       captain barbossa, long believed to be dead, ha...
2       a cryptic message from bond’s past sends him o...
3       following the death of district attorney harve...
4       john carter is a war-weary, former military ca...
                              ...                        
4798    el mariachi just wants to play his guitar and ...
4799    a newlywed couple's honeymoon is upended by th...
4800    "signed, sealed, delivered" introduces a dedic...
4801    when ambitious new york attorney sam is sent t...
4802    ever since the second grade when he first saw ...
Name: overview, Length: 4803, dtype: object

In [63]:
df['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

Remove Punctuation

In [64]:
print(df['overview'].dtype)

object


In [68]:
df['overview'] = df['overview'].astype(str)

In [65]:
import string,time
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [66]:
exclude = string.punctuation
def remove_punc(text):
  return text.translate(str.maketrans("","",exclude))

In [69]:
df['overview'].apply(remove_punc)

0       In the 22nd century a paraplegic Marine is dis...
1       Captain Barbossa long believed to be dead has ...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a warweary former military capt...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couples honeymoon is upended by the...
4800    Signed Sealed Delivered introduces a dedicated...
4801    When ambitious New York attorney Sam is sent t...
4802    Ever since the second grade when he first saw ...
Name: overview, Length: 4803, dtype: object

Removing Stopwords

In [76]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [83]:
import nltk

stopwords_words = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    new_text = []
    for word in text.split():
        if word not in stopwords_words:
            new_text.append(word)
    return ' '.join(new_text)

df['overview'] = df['overview'].apply(remove_stopwords)

In [84]:
df['overview'][0]

'In 22nd century, paraplegic Marine dispatched moon Pandora unique mission, becomes torn following orders protecting alien civilization.'